所谓自动特征工程，即是将人工特征工程的过程自动化。以 featuretools 为代表的自动特征工程在整个机器学习的端到端实践中扮演的角色.

- 主页: [Featuretools](https://docs.featuretools.com/index.html)

In [1]:
import featuretools as ft

In [2]:
# Load Mock Data
data = ft.demo.load_mock_customer()   # data 为 dict 类型 

In [4]:
data.keys()

dict_keys(['customers', 'sessions', 'transactions', 'products'])

顾客发生了多次购买会话 (session)，每次会话产生了多次交易 (transaction).

In this toy dataset, there are 3 tables. Each table is called an entity in Featuretools.

- customers: unique customers who had sessions
- sessions: unique sessions and associated attributes
- transactions: list of events in this session

In [6]:
customers_df = data['customers']
sessions_df = data['sessions']
transactions_df = data['transactions']

## 构建数据实体 (entity)

In [7]:
entities = {
    "customers": (customers_df, "customer_id"),
    "sessions": (sessions_df, "session_id", "session_start"),
    "transactions": (transactions_df, "transaction_id", "transaction_time")
}

## 指定关系

父实体与子实体的关系，通过如下四元组来定义：
`(parent_entity, parent_variable, child_entity, child_variable)`.

In [10]:
relationships = [("sessions", "session_id", "transactions", "session_id"),
                 ("customers", "customer_id", "sessions", "customer_id")]

## 深度特征合成 (Deep Feature Synthesis, DFS)

In [12]:
feature_matrix_customers, features_defs = ft.dfs(entities=entities, relationships=relationships, target_entity="customers")

# 特征工程基本概念

特征工程意味着从现有的数据中构造额外特征，这些特征通常分布在多张相关的表 (tables) 中。特征工程需要从数据中提取相关信息并将其存入单张表格中，然后被用来训练机器学习模型。

构造特征是一个非常耗时的过程，因为每个新的特征通常需要几步才能构造，特别是当使用多张表的信息时。我们可以将特征构造的操作分为两类：「转换」和「聚合」。以下通过几个例子来看看这些概念的实际应用。

- 通过从一或多列中构造新的特征，「转换」作用于单张表 (在 Python 中，表是一个 Pandas DataFrame).
- 另一方面，「聚合」是跨表实现的，并使用一对多的关联来对观测值分组，然后计算统计量。

理想情况下，我们希望有一个解决方案能够在不同表间自动执行转换和聚合操作，并将结果整合到一张表中。尽管 Pandas 是一个很好的资源，但是仍然有许多数据操作需要我们人工完成！有关人工特征工程的更多信息，请查阅《[Python Data Science Handbook](https://notebooks.azure.com/jakevdp/libraries/pythondatasciencehandbook)》.

## 特征工具

幸运的是，Feature Tools 正是我们正在找寻的解决方案。这个开源的 Python 库可以从一组相关的表中自动构造特征。特征工具基于名为「深度特征合成」的方法（参见《[Deep Feature Synthesis: Towards Automating Data Science Endeavors](http://groups.csail.mit.edu/EVO-DesignOpt/groupWebSite/uploads/Site/DSAA_DSM_2015.pdf)》），这个方法的名字听起来比其本身更高大上（这个名字源于叠加了多重特征，而不是因为使用了深度学习方法！）。

**深度特征合成**叠加多个转换和聚合操作，这在特征工具的词库中被称为**特征基元**，以便通过分布在多张表内的数据来构造新的特征。与机器学习中的大多数方法一样，这是建立在简单概念基础之上的复杂方法。通过一次学习一个构造块，我们可以很好地理解这个强大的方法。

特征工具的前两个概念的是「实体」和「实体集」。一个实体就是一张表（或是 Pandas 中的一个 DataFrame（数据框））。一个实体集是一组表以及它们之间的关联。将一个实体集看成另一种 Python 数据结构，并带有自己的方法和属性。

我们可以通过以下操作在特征工具中创建一个空的实体集：

In [14]:
import featuretools as ft
# Create new entityset
es = ft.EntitySet(id='clients')

In [15]:
现在我们添加实体。每个实体都必须有一个索引，该索引是一个包含所有唯一元素的列。也就是说，索引中的每个值只能出现在表中一次。

Entityset: clients
  Entities:
  Relationships:
    No relationships